<img src="Images/HSP2new.png" />
This Jupyter Notebook Copyright 2017 by RESPEC, INC.  All rights reserved.

# Calleg TEST NOTEBOOK for HSP$^2$ 
# Developed for Windows Plaform

This Notebook will compare the results of running HSPF and HSP$^2$ for the basic hydrology (PWATER, IWATER, and HYDR) to confirm the proper calculations of HSP$^2$.  This notebook requires HSPF to be installed and currently runs WinHSPF Lite from BASINS install C:\BASINS41\models\HSPF\bin\WinHspfLt.exe. It is setup to run using the Calleguas example provided in the repo at 

### Required Python imports  and setup

In [3]:
import os
import site

site.addsitedir(
    os.getcwd().rsplit("\\", 1)[0] + "\\"
)  # adds your path to the HSP2 software.

import shutil

import numpy as np
import pandas as pd

pd.options.display.max_rows = 18
pd.options.display.max_columns = 10
pd.options.display.float_format = (
    "{:.2f}".format
)  # display 2 digits after the decimal point

import matplotlib.pyplot as plt

import HSP2
import HSP2tools

%matplotlib inline

HSP2tools.reset_tutorial()  # make a new copy of the tutorial's data
HSP2tools.versions()  # display version information below

AttributeError: module 'HSP2tools' has no attribute 'reset_tutorial'

### Setup paths to the necessary datafiles
This assumes the calleg.uci and calleg.wdm files are located in the current working directory for this Notebook. This will create the binary output file, calleg.hbn, along with a number of other standard HSPF outputs.

In [ ]:
wdmname = "TutorialData/calleg.wdm"
uciname = "TutorialData/calleg.uci"
hdfname = "TutorialData/calleg.h5"
hbnname = "TutorialData/calleg.hbn"

## Run HSPF

Using the Basins 4.1 WinHspfLt executable to run calleg.uci.

This assumes the calleg.uci and calleg.wdm files are located in the current working directory for this Notebook.  This will create the binary output file, calleg.hbn, along with a number of other standard HSPF outputs.

In [ ]:
%time !C:\BASINS41\models\HSPF\bin\WinHspfLt.exe {uciname}

## Run HSP$^2$

This assumes the calleg.h5 file has been created. If not, the following lines can be used to create the HDF5 file (see Tutorial 4 for details.)

In [ ]:
HSP2tools.makeH5()
HSP2tools.readUCI(uciname, hdfname)
HSP2tools.ReadWDM(wdmname, hdfname)

Now run HSP$^2$ on the calleg watershed

In [ ]:
%time HSP2.run(hdfname, saveall=True)

## Determine Available Calculated Results

Now use Tim Cera's hspfbintoolbox.py to determine the available timeseries created by HSPF and stored into the HBN binary file.

In [ ]:
!ptrepack TutorialData\calleg.h5 TutorialData\packedCalleg.h5

In [ ]:
import hspfbintoolbox

In [ ]:
keys = sorted(hspfbintoolbox.catalog(hbnname).keys())
df_keys = pd.DataFrame(data=keys)
df_keys

## Automate checking IMPLNDs for SURO

Extract the keys (calculated above) for IMPLD + IWATER + SURO. For each key, compute several columns.  The final column shows the percent difference of the sum of the SURO for the entire run between HSPF and HSP2.

In [ ]:
segments = []
for operation, segment, optype, variable, freq in keys:
    if (
        str(operation) == "IMPLND"
        and str(optype) == "IWATER"
        and str(variable) == "SURO"
        and freq == 4
    ):
        segments.append(str(segment))

dfimplnd = pd.DataFrame()
for seg in segments:
    path = "IMPLND," + seg + ",IWATER,SURO"
    hspf, units_flag = HSP2tools.get_HBNdata(hbnname, path)
    hspf = hspf["M"].values

    path = "/RESULTS/IMPLND" + "_I" + "{:0>3s}".format(str(seg)) + "/IWATER"
    hsp2 = pd.read_hdf(hdfname, path)["SURO"]
    hsp2 = hsp2.resample("M").sum().values

    dfimplnd.at[seg, "Max Diff"] = (hspf - hsp2).max()
    dfimplnd.at[seg, "Sum of HSPF"] = hspf.sum()
    dfimplnd.at[seg, "Sum of HSP2"] = hsp2.sum()
    dfimplnd.at[seg, "%diff of Sum"] = 100.0 * (hspf.sum() - hsp2.sum()) / hspf.sum()
    dfimplnd.at[seg, "abs(%diff of Sum)"] = (
        100.0 * abs(hspf.sum() - hsp2.sum()) / hspf.sum()
    )

dfimplnd = dfimplnd.sort_values(by=["abs(%diff of Sum)"])
dfimplnd

Look at the statistics for the percent difference column

In [ ]:
dfimplnd["%diff of Sum"].describe()

### Now look at the worst case
The IMPLND segments are ordered in assending "abs(%diff of Sum)", so the last entry is the worst case (by this measure.)

In [ ]:
ils = dfimplnd.index[-1]
print("WORST IMPLND SEGMENT IS ", ils)
print("%diff of the total SURO sum of", dfimplnd.loc[ils, "%diff of Sum"])

### Define a function to read HSPF and HSP2 data, and plot together for IMPLND

In [ ]:
def imp(ils, name, how="sum"):
    # Use Tim Cera's HBN reader to get the HSPF data
    path = "IMPLND," + str(ils) + ",IWATER," + name
    hspf, units_flag = HSP2tools.get_HBNdata(hbnname, path)
    # There may be both monthly and annual timeseries available. Get the monthly timeseries.
    hspf = hspf["M"]

    # Now read the corresponding HSP2 data and comvert to monthly
    path = "/RESULTS/IMPLND" + "_I" + "{:0>3s}".format(str(ils)) + "/IWATER"
    hsp2 = pd.read_hdf(hdfname, path)
    if how == "sum":
        hsp2 = hsp2.resample("M").sum()
    elif how == "last":
        hsp2 = hsp2.resample("M").last()

    hsp2 = hsp2[name]

    plt.figure(figsize=(16, 8))
    plt.plot(hspf.index, hspf, label="HSPF", color="r")
    plt.plot(hsp2.index, hsp2, label="HSP2", color="b", linestyle="--")
    plt.legend()
    plt.title("IMPLND " + "I" + "{:0>3s}".format(str(ils)) + ", IWATER " + name)

    return hspf, hsp2

#### IMPLND IWATER SURO, Monthly

In [ ]:
hspf, hsp2 = imp(ils, "SURO", "sum")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### IMPLND IWATER IMPEV, Monthly

In [ ]:
hspf, hsp2 = imp(ils, "IMPEV")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### IMPLND IWATER PET, Monthly

In [ ]:
hspf, hsp2 = imp(ils, "PET")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### IMPLND IWATER RETS, Monthly

In [ ]:
hspf, hsp2 = imp(ils, "RETS", "last")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### IMPLND IWATER SUPY, Monthly

In [ ]:
hspf, hsp2 = imp(ils, "SUPY")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### IMPLND IWATER SURS, Monthly

In [ ]:
hspf, hsp2 = imp(ils, "SURS", "last")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

## Automate checking PERLNDs for PERO

In [ ]:
# With Large HBNs this currently takes a while (need to speed up the HBN Read)
segments = []
for operation, segment, optype, variable, freq in keys:
    if (
        str(operation) == "PERLND"
        and str(optype) == "PWATER"
        and str(variable) == "PERO"
        and freq == 4
    ):
        segments.append(str(segment))

dfperlnd = pd.DataFrame()
for seg in segments:
    path = "PERLND," + seg + ",PWATER,PERO"
    hspf, units_flag = HSP2tools.get_HBNdata(hbnname, path)
    hspf = hspf["M"].values

    path = "RESULTS/PERLND" + "_P" + "{:0>3s}".format(str(seg)) + "/PWATER"
    hsp2 = pd.read_hdf(hdfname, path)["PERO"]
    hsp2 = hsp2.resample("M").sum().values

    dfperlnd.at[seg, "Max Diff"] = (hspf - hsp2).max()
    dfperlnd.at[seg, "Sum of HSPF"] = hspf.sum()
    dfperlnd.at[seg, "Sum of HSP2"] = hsp2.sum()
    dfperlnd.at[seg, "%diff of Sum"] = 100.0 * (hspf.sum() - hsp2.sum()) / hspf.sum()
    dfperlnd.at[seg, "abs(%diff of Sum)"] = (
        100.0 * abs(hspf.sum() - hsp2.sum()) / hspf.sum()
    )

dfperlnd = dfperlnd.sort_values(by=["abs(%diff of Sum)"])
dfperlnd

In [ ]:
dfperlnd["%diff of Sum"].describe()

The PERLND segments are ordered in assending "abs(%diff of Sum)", so the last entry is the worst case (by this measure.)

In [ ]:
pls = dfperlnd.index[-1]
print("WORST PERLND SEGMENT IS ", pls)
print("%diff of the total PERO sum of", dfperlnd.loc[pls, "%diff of Sum"])

### Define routine to read HSPF and HSP2 data and plot together

In [ ]:
def per(pls, name, how="sum"):
    # Use Tim Cera's HBN reader to get the HSPF data
    path = "PERLND," + str(pls) + ",PWATER," + name
    hspf, units_flag = HSP2tools.get_HBNdata(hbnname, path)
    # There may be both monthly and annual timeseries available. Get the monthly timeseries.
    hspf = hspf["M"]

    # Now read the corresponding HSP2 data and comvert to monthly
    path = "/RESULTS/PERLND" + "_P" + "{:0>3s}".format(str(pls)) + "/PWATER"
    if how == "sum":
        hsp2 = pd.read_hdf(hdfname, path)[name].resample("M").sum()
    elif how == "last":
        hsp2 = pd.read_hdf(hdfname, path)[name].resample("M").last()

    plt.figure(figsize=(16, 8))
    plt.plot(hspf.index, hspf, label="HSPF", color="r")
    plt.plot(hsp2.index, hsp2, label="HSP2", color="b", linestyle="--")
    plt.legend()
    plt.title("PERLND " + "P" + "{:0>3s}".format(str(pls)) + ", PWATER " + name)

    return hspf, hsp2

#### PERLND PWATER AGWO

In [ ]:
hspf, hsp2 = per(pls, "AGWO")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER BASET

In [ ]:
hspf, hsp2 = per(pls, "BASET")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

##### PERLND PWATER CEPE

In [ ]:
hspf, hsp2 = per(pls, "CEPE")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER IFWI

In [ ]:
hspf, hsp2 = per(pls, "IFWI")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER IFWO

In [ ]:
hspf, hsp2 = per(pls, "IFWO")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER IGWI

In [ ]:
hspf, hsp2 = per(pls, "IGWI")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER INFIL

In [ ]:
hspf, hsp2 = per(pls, "INFIL")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER LZET

In [ ]:
hspf, hsp2 = per(pls, "LZET")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER PERC

In [ ]:
hspf, hsp2 = per(pls, "PERC")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER PERO

In [ ]:
hspf, hsp2 = per(pls, "PERO")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER PERS

In [ ]:
hspf, hsp2 = per(pls, "PERS", "last")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER PET

In [ ]:
hspf, hsp2 = per(pls, "PET")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER SUPY

In [ ]:
hspf, hsp2 = per(pls, "SUPY")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER SURO

In [ ]:
hspf, hsp2 = per(pls, "SURO")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER TAET

In [ ]:
hspf, hsp2 = per(pls, "TAET")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER UZET

In [ ]:
hspf, hsp2 = per(pls, "UZET")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### PERLND PWATER UZI

In [ ]:
hspf, hsp2 = per(pls, "UZI")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

## RCHRES

### Automate checking RCHRESs for ROVOL

In [ ]:
# With Large HBNs this currently takes a while (need to speed up the HBN Read)
segments = []
for operation, segment, optype, variable, freq in keys:
    if (
        str(operation) == "RCHRES"
        and str(optype) == "HYDR"
        and str(variable) == "ROVOL"
        and freq == 4
    ):
        segments.append(str(segment))

dfrchres = pd.DataFrame()
for seg in segments:
    string = "RCHRES," + seg + ",HYDR,ROVOL"
    hspf, units_flag = HSP2tools.get_HBNdata(hbnname, string)
    hspf = hspf["M"].values

    string = "RESULTS/RCHRES" + "_R" + "{:0>3s}".format(str(seg)) + "/HYDR"
    hsp2 = pd.read_hdf(hdfname, string)["ROVOL"]
    hsp2 = hsp2.resample("M").sum().values

    dfrchres.at[seg, "Max Diff"] = (hspf - hsp2).max()
    dfrchres.at[seg, "Sum of HSPF"] = hspf.sum()
    dfrchres.at[seg, "Sum of HSP2"] = hsp2.sum()
    dfrchres.at[seg, "%diff of Sum"] = 100.0 * (hspf.sum() - hsp2.sum()) / hspf.sum()
    dfrchres.at[seg, "abs(%diff of Sum)"] = (
        100.0 * abs(hspf.sum() - hsp2.sum()) / hspf.sum()
    )

dfrchres = dfrchres.sort_values(by=["abs(%diff of Sum)"])
dfrchres

In [ ]:
dfrchres["%diff of Sum"].describe()

The RCHRES segments are ordered in assending "abs(%diff of Sum)", so the last entry is the worst case (by this measure.)

In [ ]:
rid = dfrchres.index[-1]
print("WORST RCHRES SEGMENT IS ", rid)
print("%diff of the total ROVOL sum of", dfrchres.loc[rid, "%diff of Sum"])

### Define routine to read HSPF and HSP2, plot together for RCHRES

In [ ]:
def rch(rid, name, how="sum"):
    # Use Tim Cera's HBN reader to get the HSPF data
    path = "RCHRES," + str(rid) + ",HYDR," + name
    hspf, units_flag = HSP2tools.get_HBNdata(hbnname, path)
    # There may be both monthly and annual timeseries available. Get the monthly timeseries.
    hspf = hspf["M"]

    # Now read the corresponding HSP2 data and comvert to monthly
    path = "/RESULTS/RCHRES" + "_R" + "{:0>3s}".format(str(rid)) + "/HYDR"
    if how == "sum":
        hsp2 = pd.read_hdf(hdfname, path)[name].resample("M").sum()
    elif how == "last":
        hsp2 = pd.read_hdf(hdfname, path)[name].resample("M").last()

    plt.figure(figsize=(16, 8))
    plt.plot(hspf.index, hspf, label="HSPF", color="r")
    plt.plot(hsp2.index, hsp2, label="HSP2", color="b", linestyle="--")
    plt.legend()
    plt.title("RCHRES " + "R" + "{:0>3s}".format(str(rid)) + ", HYDR " + name)

    return hspf, hsp2

#### RCHRES HYDR IVOL

In [ ]:
hspf, hsp2 = rch(rid, "IVOL")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### RCHRES HYDR PRSUPY

In [ ]:
hspf, hsp2 = rch(rid, "PRSUPY")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### RCHRES HYDR ROVOL

In [ ]:
hspf, hsp2 = rch(rid, "ROVOL")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### RCHRES HYDR VOL

In [ ]:
hspf, hsp2 = rch(rid, "VOL", "last")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])

#### RCHRES HYDR VOLEV

In [ ]:
hspf, hsp2 = rch(rid, "VOLEV")

In [ ]:
plt.scatter(hspf, hsp2)
top = 1.05 * max(hspf.max(), hsp2.max())
plt.plot([0.0, top], [0.0, top])